# VacationPy
----

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** if you having trouble displaying the maps, try running `jupyter nbextension enable --py gmaps` in your environment and retry.

To complete this part of the assignment,you will need to do the following:

* Create a heat map that displays the humidity for every city from Part I.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_path = "../WeatherPy/output_files/cleaned_weather.csv"
weather_data = pd.read_csv(weather_path)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,77.81,67,12,9.48,PF,1612768182
1,Jamestown,42.0970,-79.2353,10.99,85,90,3.44,US,1612767939
2,San Vicente,-26.6167,-54.1333,60.53,89,0,3.87,AR,1612768183
3,Hofn,64.2539,-15.2082,36.97,79,100,13.69,IS,1612768183
4,Mahébourg,-20.4081,57.7000,86.00,62,40,13.80,MU,1612768183
...,...,...,...,...,...,...,...,...,...
548,Pathein,16.7833,94.7333,86.40,30,9,15.01,MM,1612768330
549,Chapais,49.7834,-74.8492,-13.00,84,1,3.44,CA,1612768330
550,Skibotn,69.3913,20.2679,28.40,64,75,8.05,NO,1612768330
551,Honningsvåg,70.9821,25.9704,24.80,86,90,28.77,NO,1612768331


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]].astype(float)

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type = "HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,max_intensity = 100,point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

##### Conditions used:
  * A max temperature lower than 80 degrees but higher than 70 for the right amount of sunshine.

  * Wind speed less than 5 mph.

  * 5% of cloudiness for some cloud watching

In [6]:
weather_data["Max Temp"]

0      77.81
1      10.99
2      60.53
3      36.97
4      86.00
       ...  
548    86.40
549   -13.00
550    28.40
551    24.80
552    32.00
Name: Max Temp, Length: 553, dtype: float64

In [7]:
ideal_cities = weather_data.loc[(weather_data["Max Temp"] < 80 )& (weather_data["Max Temp"] > 65 ) & (weather_data["Wind Speed"] < 5 ) & (weather_data["Cloudiness"] < 5 ) ]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Balancán,17.8000,-91.5333,67.39,95,0,0.89,MX,1612768184
139,Maceió,-9.6658,-35.7353,71.60,100,0,4.61,BR,1612768222
191,Pisco,-13.7000,-76.2167,71.60,83,0,3.44,PE,1612768195
235,Rosario,-32.9468,-60.6393,75.00,88,0,2.30,AR,1612768248
238,Córdoba,-31.4135,-64.1811,73.00,94,0,3.44,AR,1612767995
269,Wahiawā,21.5028,-158.0236,71.01,88,1,3.44,US,1612768258
282,Caucayá,-0.1934,-74.7819,71.91,95,4,1.95,CO,1612768262
350,Qena,26.1615,32.7183,66.20,59,0,4.61,EG,1612768280
354,Gumlā,23.0500,84.5500,78.60,17,0,3.38,IN,1612768281
360,Manacapuru,-3.2997,-60.6206,73.40,94,0,3.74,BR,1612768200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_cities

coordinates = []
#hotel_df["Hotel Name"]

In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\tiffa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tiffa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,Balancán,17.8000,-91.5333,67.39,95,0,0.89,MX,1612768184,Hotel Balam Ku
139,Maceió,-9.6658,-35.7353,71.60,100,0,4.61,BR,1612768222,Jatiúca Hotel & Resort
191,Pisco,-13.7000,-76.2167,71.60,83,0,3.44,PE,1612768195,PiscoMar Peru
235,Rosario,-32.9468,-60.6393,75.00,88,0,2.30,AR,1612768248,Ros Tower Hotel
238,Córdoba,-31.4135,-64.1811,73.00,94,0,3.44,AR,1612767995,Virreinato
269,Wahiawā,21.5028,-158.0236,71.01,88,1,3.44,US,1612768258,The Inn At Schofield
282,Caucayá,-0.1934,-74.7819,71.91,95,4,1.95,CO,1612768262,Hotel San Pedro Plaza
350,Qena,26.1615,32.7183,66.20,59,0,4.61,EG,1612768280,Basma Hotel
354,Gumlā,23.0500,84.5500,78.60,17,0,3.38,IN,1612768281,Hotel Nipra
360,Manacapuru,-3.2997,-60.6206,73.40,94,0,3.74,BR,1612768200,TBS HOTEL


In [12]:
# Add marker layer ontop of heat map with Hotel Name, City, and Country
locations = hotel_df[["Lat","Lng"]].astype(float)
info_box = []
cleaned_hotel_df = hotel_df.dropna()

for index, row in hotel_df.iterrows():
    info_box.append(f"Hotel: {row['Hotel Name']} City: {row['City']} Country: {row['Country']}")
print(info_box)

['Hotel: Hotel Balam Ku City: Balancán Country: MX', 'Hotel: Jatiúca Hotel & Resort City: Maceió Country: BR', 'Hotel: PiscoMar Peru City: Pisco Country: PE', 'Hotel: Ros Tower Hotel City: Rosario Country: AR', 'Hotel: Virreinato City: Córdoba Country: AR', 'Hotel: The Inn At Schofield City: Wahiawā Country: US', 'Hotel: Hotel San Pedro Plaza City: Caucayá Country: CO', 'Hotel: Basma Hotel City: Qena Country: EG', 'Hotel: Hotel Nipra City: Gumlā Country: IN', 'Hotel: TBS HOTEL City: Manacapuru Country: BR', 'Hotel: nan City: Napāsar Country: IN', 'Hotel: nan City: Ipixuna Country: BR', 'Hotel: Hotel Villa Amazônia City: Manaus Country: BR', 'Hotel: Sheraton Salta Hotel City: Salta Country: AR', 'Hotel: nan City: Tacuatí Country: PY', 'Hotel: Hôtel le Parisien City: Kétou Country: BJ', 'Hotel: Hotel Aalo City: Along Country: IN', 'Hotel: nan City: Barentu Country: ER']


In [14]:
# Assign the marker layer to a variable
marker = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)',
                                scale=2, info_box_content = [x for x in info_box])
# Add the layer to the map
                      
fig.add_layer(marker)
fig
# Display figure

Figure(layout=FigureLayout(height='420px'))